In [1]:
import pandas as pd
from datetime import datetime
import pycountry

In [7]:
# Leer el archivo Excel
df = pd.read_excel('data/emdat-data.xlsx')

# Crear la columna 'duration' calculando los días entre las fechas de inicio y fin
def calculate_duration(row):
    try:
        # Crear fechas de inicio y fin
        start_year = row['Start Year']
        start_month = row['Start Month'] if pd.notna(row['Start Month']) else 1
        start_day = row['Start Day'] if pd.notna(row['Start Day']) else 1
        
        end_year = row['End Year']
        end_month = row['End Month'] if pd.notna(row['End Month']) else 12
        end_day = row['End Day'] if pd.notna(row['End Day']) else 31
        
        # Si alguno de los años no está disponible, retornar None
        if pd.isna(start_year) or pd.isna(end_year):
            return None
        
        # Crear objetos datetime
        start_date = datetime(int(start_year), int(start_month), int(start_day))
        end_date = datetime(int(end_year), int(end_month), int(end_day))
        
        # Calcular la duración en días
        duration = (end_date - start_date).days
        
        return duration if duration >= 0 else 0
    except:
        return None
    
# Crear columnas Start Date y End Date
def create_date_column(row, prefix):
    """
    Crea una fecha a partir de Year, Month, Day con valores por defecto si faltan
    """
    try:
        year = row[f'{prefix} Year']
        month = row[f'{prefix} Month'] if pd.notna(row[f'{prefix} Month']) else 1
        day = row[f'{prefix} Day'] if pd.notna(row[f'{prefix} Day']) else 1
        
        if pd.isna(year):
            return pd.NaT
        
        return pd.to_datetime(f"{int(year)}-{int(month):02d}-{int(day):02d}", errors='coerce')
    except:
        return pd.NaT

def iso_to_country_name(iso_code):
    """
    Convierte un código ISO-3 a nombre de país
    """
    if pd.isna(iso_code):
        return None
    
    try:
        country = pycountry.countries.get(alpha_3=iso_code)
        return country.name if country else None
    except:
        return None

# Aplicar la función para crear las columnas de fecha
df['Start Date'] = df.apply(lambda row: create_date_column(row, 'Start'), axis=1)
df['End Date'] = df.apply(lambda row: create_date_column(row, 'End'), axis=1)

# Guardar el dataset actualizado

# Aplicar la función para crear la columna duration
df['Duration'] = df.apply(calculate_duration, axis=1)

# Aplicar la conversión
df['Country Name'] = df['ISO'].apply(iso_to_country_name)

# Mostrar resultados
print(f"Country Name creado - Valores no nulos: {df['Country Name'].notna().sum()} de {len(df)}")
print("\nComparación ISO vs Country Name:")
print(df[['ISO', 'Country', 'Country Name']].head(20))

# Eliminar las columnas especificadas
columns_to_drop = ['DisNo.', 
                'Historic', 
                'Classification Key', 
                'Disaster Group', 
                'External IDs', 
                'Admin Units', 
                'Latituede', 
                'Longitude', 
                'Latitude', 
                'Associated Types', 
                'GADM Admin Units', 
                'OFDA/BHA Response', 
                'Appeal', 
                'CPI', 
                'Declaration', 
                'River Basin', 
                "Total Damage, Adjusted ('000 US$)", 
                "Insured Damage, Adjusted ('000 US$)", 
                "Reconstruction Costs, Adjusted ('000 US$)", 
                "Disaster Subtype", "Origin", "Location", 
                "Reconstruction Costs ('000 US$)", 
                "AID Contribution ('000 US$)", 
                "Insured Damage ('000 US$)",
                "AID Contribution ('000 US$)", 
                "Reconstruction Costs ('000 US$)",
                "Insured Damage ('000 US$)", 
                "Start Month", 
                "Start Day", 
                "End Year", 
                "End Month", 
                "End Day",
                "Country",
                "Duration",
                "Entry Date",
                "Last Update",
                "Total Damage ('000 US$)",
                "No. Homeless",
                "No. Affected",
                "No. Injured",
                "Magnitude Scale",
                "Magnitude"
            ]

# Verificar qué columnas existen antes de eliminarlas
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

if existing_columns_to_drop:
    df = df.drop(columns=existing_columns_to_drop)
    print(f"Columnas eliminadas: {existing_columns_to_drop}")
else:
    print("No se encontraron columnas para eliminar")

mapping = {
    "Mass movement (dry)": "Mass Movement",
    "Mass movement (wet)": "Mass Movement",
    "Glacial lake outburst flood": "Glacial Flood",
    "Animal incident": "Animal Incident",
    "Extreme temperature": "Extreme Temperature",
    "Volcanic activity": "Volcanic Activity"
}

df["Disaster Type"] = df["Disaster Type"].replace(mapping)
# Crear columna Country Name a partir del código ISO

# Guardar el dataset procesado
df.to_csv('data/emdat-data-processed.csv', index=False)

Country Name creado - Valores no nulos: 10601 de 10622

Comparación ISO vs Country Name:
    ISO                           Country  \
0   BRA                            Brazil   
1   USA          United States of America   
2   RWA                            Rwanda   
3   USA          United States of America   
4   NGA                           Nigeria   
5   COD  Democratic Republic of the Congo   
6   POL                            Poland   
7   PAN                            Panama   
8   IDN                         Indonesia   
9   AFG                       Afghanistan   
10  GTM                         Guatemala   
11  CRI                        Costa Rica   
12  MOZ                        Mozambique   
13  ZAF                      South Africa   
14  NPL                             Nepal   
15  NIC                         Nicaragua   
16  IDN                         Indonesia   
17  YEM                             Yemen   
18  GMB                            Gambia   
19  NPL    

In [9]:
study_data = pd.read_csv('data/emdat-data-processed.csv')

In [10]:
def load_study_data(file_path):
    """
    Load study dataset from a CSV file.

    Parameters:
    file_path (str): The path to the CSV file containing the study data.

    Returns:
    pd.DataFrame: A DataFrame containing the loaded study data.
    """
    try:
        # Detectar si es CSV o Excel por la extensión
        if file_path.endswith('.csv'):
            data = pd.read_csv(file_path)
        else:
            data = pd.read_excel(file_path)
        return data
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None
    except pd.errors.EmptyDataError:
        print("Error: The file is empty.")
        return None
    except pd.errors.ParserError:
        print("Error: There was a parsing error while reading the file.")
        return None
    
def analyze_missing_values(data):
    """
    Analyze missing values (NaN) in the dataset.

    Parameters:
    data (pd.DataFrame): The DataFrame to analyze.

    Returns:
    pd.DataFrame: A DataFrame with missing value statistics per column.
    """
    if data is None:
        return None
    
    missing_stats = pd.DataFrame({
        'Column': data.columns,
        'Missing_Count': data.isnull().sum().values,
        'Missing_Percentage': (data.isnull().sum().values / len(data) * 100).round(2),
        'Data_Type': data.dtypes.values
    })
    
    # Ordenar por cantidad de valores faltantes
    missing_stats = missing_stats.sort_values('Missing_Count', ascending=False)
    
    return missing_stats


def get_basic_statistics(data):
    """
    Get basic statistics for numerical columns.

    Parameters:
    data (pd.DataFrame): The DataFrame to analyze.

    Returns:
    pd.DataFrame: Statistics for numerical columns.
    """
    if data is None:
        return None
    
    return data.describe()


def analyze_data_types(data):
    """
    Analyze data types in the dataset.

    Parameters:
    data (pd.DataFrame): The DataFrame to analyze.

    Returns:
    dict: Dictionary with counts of each data type.
    """
    if data is None:
        return None
    
    dtype_counts = data.dtypes.value_counts()
    return dtype_counts


def get_unique_values_summary(data, max_unique=50):
    """
    Get summary of unique values for each column.

    Parameters:
    data (pd.DataFrame): The DataFrame to analyze.
    max_unique (int): Maximum number of unique values to display details.

    Returns:
    pd.DataFrame: Summary of unique values per column.
    """
    if data is None:
        return None
    
    unique_summary = pd.DataFrame({
        'Column': data.columns,
        'Unique_Count': [data[col].nunique() for col in data.columns],
        'Sample_Values': [str(data[col].dropna().unique()[:3].tolist()) if data[col].nunique() <= max_unique 
                         else f"{data[col].nunique()} unique values" 
                         for col in data.columns]
    })
    
    return unique_summary


def summarize_study_data(data):
    """
    Summarize the study dataset.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the study data.

    Returns:
    dict: A summary of the dataset including number of rows, columns, and column names.
    """
    if data is None:
        return None

    summary = {
        'num_rows': data.shape[0],
        'num_columns': data.shape[1],
        'column_names': data.columns.tolist(),
        'total_missing': data.isnull().sum().sum(),
        'memory_usage': data.memory_usage(deep=True).sum() / 1024**2  # En MB
    }
    return summary

# Cargar el dataset procesado' 
print("="*80)
print("ANÁLISIS DEL DATASET")
print("="*80)
    
if study_data is not None:
    # 1. Resumen básico
    print("\n" + "="*80)
    print("1. RESUMEN GENERAL")
    print("="*80)
    summary = summarize_study_data(study_data)
    print(f"Número de filas: {summary['num_rows']:,}")
    print(f"Número de columnas: {summary['num_columns']}")
    print(f"Total de valores faltantes: {summary['total_missing']:,}")
    print(f"Uso de memoria: {summary['memory_usage']:.2f} MB")
    
# 2. Análisis de valores faltantes
    print("\n" + "="*80)
    print("2. ANÁLISIS DE VALORES FALTANTES (NaN)")
    print("="*80)
    missing_analysis = analyze_missing_values(study_data)
    print("\nColumnas con valores faltantes:")
    print(missing_analysis[missing_analysis['Missing_Count'] > 0].to_string(index=False))
        
    # 3. Tipos de datos
    print("\n" + "="*80)
    print("3. DISTRIBUCIÓN DE TIPOS DE DATOS")
    print("="*80)
    dtype_analysis = analyze_data_types(study_data)
    print(dtype_analysis)

    # 4. Estadísticas básicas
    print("\n" + "="*80)
    print("4. ESTADÍSTICAS DESCRIPTIVAS (Columnas Numéricas)")
    print("="*80)
    stats = get_basic_statistics(study_data)
    print(stats)
        
    # 5. Valores únicos
    print("\n" + "="*80)
    print("5. RESUMEN DE VALORES ÚNICOS")
    print("="*80)
    unique_summary = get_unique_values_summary(study_data)
    print(unique_summary.to_string(index=False))
        
    # 6. Primeras filas
    print("\n" + "="*80)
    print("6. PRIMERAS 5 FILAS DEL DATASET")
    print("="*80)
    print(study_data.head())
        
    # 7. Información de columnas
    print("\n" + "="*80)
    print("7. INFORMACIÓN DETALLADA DE COLUMNAS")
    print("="*80)
    study_data.info()
else:
    print("No se pudo cargar el dataset.")

ANÁLISIS DEL DATASET

1. RESUMEN GENERAL
Número de filas: 10,622
Número de columnas: 12
Total de valores faltantes: 13,008
Uso de memoria: 6.06 MB

2. ANÁLISIS DE VALORES FALTANTES (NaN)

Columnas con valores faltantes:
        Column  Missing_Count  Missing_Percentage Data_Type
    Event Name           7998               75.30    object
  Total Deaths           3006               28.30   float64
Total Affected           1983               18.67   float64
  Country Name             21                0.20    object

3. DISTRIBUCIÓN DE TIPOS DE DATOS
object     9
float64    2
int64      1
Name: count, dtype: int64

4. ESTADÍSTICAS DESCRIPTIVAS (Columnas Numéricas)
         Start Year   Total Deaths  Total Affected
count  10622.000000    7616.000000    8.639000e+03
mean    2012.217379     222.251838    5.615388e+05
std        7.671002    3997.092313    6.205882e+06
min     2000.000000       1.000000    1.000000e+00
25%     2005.000000       4.000000    6.120000e+02
50%     2012.000000    

In [11]:
print(study_data["Disaster Type"].unique())

['Flood' 'Wildfire' 'Drought' 'Mass Movement' 'Volcanic Activity'
 'Earthquake' 'Storm' 'Epidemic' 'Extreme Temperature' 'Infestation'
 'Glacial Flood' 'Animal Incident' 'Impact']


In [12]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv('data/emdat-data-processed.csv')

# Diccionario de mapeo para estandarizar nombres de países
country_name_mapping = {
    "Congo, The Democratic Republic of the": "Democratic Republic of the Congo",
    "Iran, Islamic Republic of": "Iran",
    "Tanzania, United Republic of": "Tanzania",
    "Korea, Democratic People's Republic of": "North Korea",
    "Korea, Republic of": "South Korea",
    "Lao People's Democratic Republic": "Laos",
    "Venezuela, Bolivarian Republic of": "Venezuela",
    "Moldova, Republic of": "Moldova",
    "Russian Federation": "Russia",
    "Syrian Arab Republic": "Syria",
    "Viet Nam": "Vietnam",
    "Bolivia, Plurinational State of": "Bolivia",
    "Macedonia, The Former Yugoslav Republic of": "North Macedonia",
    "Czechia": "Czech Republic",
    "Cabo Verde": "Cape Verde",
    "Cote d'Ivoire": "Ivory Coast",
    "Côte d'Ivoire": "Ivory Coast",
    "Türkiye": "Turkey",
    "Brunei Darussalam": "Brunei",
    "Micronesia, Federated States of": "Micronesia",
    "Saint Vincent and the Grenadines": "St. Vincent and the Grenadines",
    "Saint Lucia": "St. Lucia",
    "Saint Kitts and Nevis": "St. Kitts and Nevis",
    "Sao Tome and Principe": "São Tomé and Príncipe",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "United States of America": "United States"
}

# Aplicar el mapeo de nombres
df['Country Name'] = df['Country Name'].replace(country_name_mapping)

# Guardar el archivo procesado
df.to_csv('data/emdat-data-processed.csv', index=False)

print("Nombres de países estandarizados correctamente!")
print(f"\nTotal de filas procesadas: {len(df)}")
print(f"\nPaíses únicos después de la estandarización: {df['Country Name'].nunique()}")
print("\nPrimeros países únicos:")
print(df['Country Name'].unique()[:20])

Nombres de países estandarizados correctamente!

Total de filas procesadas: 10622

Países únicos después de la estandarización: 217

Primeros países únicos:
['Brazil' 'United States' 'Rwanda' 'Nigeria'
 'Democratic Republic of the Congo' 'Poland' 'Panama' 'Indonesia'
 'Afghanistan' 'Guatemala' 'Costa Rica' 'Mozambique' 'South Africa'
 'Nepal' 'Nicaragua' 'Yemen' 'Gambia' 'Mongolia' 'China' 'Colombia']


In [13]:
# Definir qué subregiones pertenecen a América del Norte y América del Sur
north_america_subregions = ['Northern America']
south_america_subregions = ['Latin America and the Caribbean']

# Función para reclasificar Americas
def reclassify_americas(row):
    if row['Region'] == 'Americas':
        if row['Subregion'] in north_america_subregions:
            return 'North America'
        elif row['Subregion'] in south_america_subregions:
            return 'South America'
    return row['Region']

# Aplicar la reclasificación
study_data['Region'] = study_data.apply(reclassify_americas, axis=1)

# Verificar los cambios
print("Regiones únicas después de la transformación:")
print(study_data['Region'].unique())

print("\nConteo de desastres por región:")
print(study_data['Region'].value_counts())

# Verificar qué subregiones se transformaron
print("\nVerificación de transformación:")
print(study_data[study_data['Region'].isin(['North America', 'South America'])][['Region', 'Subregion']].drop_duplicates())
study_data = study_data.drop(columns=['Subregion'])
# Guardar el dataset actualizado
study_data.to_csv('data/emdat-data-processed.csv', index=False)
print("\nDataset actualizado guardado.")

Regiones únicas después de la transformación:
['South America' 'North America' 'Africa' 'Europe' 'Asia' 'Oceania']

Conteo de desastres por región:
Region
Asia             4161
Africa           2225
South America    1738
Europe           1390
North America     722
Oceania           386
Name: count, dtype: int64

Verificación de transformación:
          Region                        Subregion
0  South America  Latin America and the Caribbean
1  North America                 Northern America

Dataset actualizado guardado.


In [29]:
# Crear dataset con conteo de desastres por país (usando código ISO)
disasters_by_country = study_data.groupby(['ISO', 'Country Name', 'Region', 'Disaster Type', 'Disaster Subgroup']).size().reset_index(name='Disaster_Count')

# Ordenar por cantidad de desastres (de mayor a menor)
disasters_by_country = disasters_by_country.sort_values('Disaster_Count', ascending=False)

# Guardar el dataset
disasters_by_country.to_csv('disasters_by_country.csv', index=False)

print("Dataset de desastres por país creado:")
print(f"Total de países: {len(disasters_by_country)}")
print(f"\n{disasters_by_country.head(20)}")

# Verificar la distribución por región
print("\nConteo por región:")
print(disasters_by_country.groupby('Region')['Disaster_Count'].sum().sort_values(ascending=False))

Dataset de desastres por país creado:
Total de países: 1000

     ISO   Country Name         Region Disaster Type Disaster Subgroup  \
942  USA  United States  North America         Storm    Meteorological   
153  CHN          China           Asia         Flood      Hydrological   
158  CHN          China           Asia         Storm    Meteorological   
383  IDN      Indonesia           Asia         Flood      Hydrological   
711  PHL    Philippines           Asia         Storm    Meteorological   
392  IND          India           Asia         Flood      Hydrological   
940  USA  United States  North America         Flood      Hydrological   
112  BRA         Brazil  South America         Flood      Hydrological   
150  CHN          China           Asia    Earthquake       Geophysical   
708  PHL    Philippines           Asia         Flood      Hydrological   
396  IND          India           Asia         Storm    Meteorological   
967  VNM       Viet Nam           Asia         Stor

In [31]:
import pandas as pd
import json
import os

# Leer el CSV
df = pd.read_csv('data/disasters_by_country.csv')

# Lista de archivos GeoJSON
geojson_files = [
    'data/geometria/africa.geojson',
    'data/geometria/asia.geojson',
    'data/geometria/europe.geojson',
    'data/geometria/north_america.geojson',
    'data/geometria/oceania.geojson',
    'data/geometria/south_america.geojson'
]

# Crear un diccionario de mapeo global basado en códigos ISO
country_mapping = {}

# Leer todos los GeoJSON y construir el mapeo
for geojson_file in geojson_files:
    if os.path.exists(geojson_file):
        with open(geojson_file, 'r', encoding='utf-8') as f:
            geojson = json.load(f)
            
        for feature in geojson['features']:
            props = feature['properties']
            iso_a3 = props.get('iso_a3')
            standard_name = props.get('name')
            
            if iso_a3 and standard_name:
                country_mapping[iso_a3] = standard_name

print(f"✓ Mapeo creado con {len(country_mapping)} países")

# Aplicar el mapeo usando los códigos ISO
for iso_code, standard_name in country_mapping.items():
    df.loc[df['ISO'] == iso_code, 'Country Name'] = standard_name

# Guardar el archivo actualizado
df.to_csv('data/disasters_by_country.csv', index=False)

print("✓ Nombres de países estandarizados exitosamente")
print(f"\nTotal de países únicos: {df['Country Name'].nunique()}")
print("\n✓ Primeros 30 países estandarizados:")
unique_countries = df[['ISO', 'Country Name', 'Region']].drop_duplicates().sort_values('Country Name')
print(unique_countries.head(30).to_string(index=False))

# Verificar si hay países sin mapear
unmapped = df[~df['ISO'].isin(country_mapping.keys())]['Country Name'].unique()
if len(unmapped) > 0:
    print(f"\n⚠ Advertencia: {len(unmapped)} países sin mapear:")
    print(unmapped[:10])

✓ Mapeo creado con 160 países
✓ Nombres de países estandarizados exitosamente

Total de países únicos: 217

✓ Primeros 30 países estandarizados:
ISO                    Country Name        Region
AFG                     Afghanistan          Asia
ALB                         Albania        Europe
DZA                         Algeria        Africa
ASM                  American Samoa       Oceania
AGO                          Angola        Africa
AIA                        Anguilla South America
ATG             Antigua and Barbuda South America
ARG                       Argentina South America
ARM                         Armenia          Asia
AUS                       Australia       Oceania
AUT                         Austria        Europe
AZE                      Azerbaijan          Asia
BHS                         Bahamas South America
BGD                      Bangladesh          Asia
BRB                        Barbados South America
BLR                         Belarus        Europe
BEL  

In [32]:
# IMPORTANTE: Primero estandarizar el archivo emdat-data-processed.csv
import pandas as pd
import json
import os

# Lista de archivos GeoJSON
geojson_files = [
    'data/geometria/africa.geojson',
    'data/geometria/asia.geojson',
    'data/geometria/europe.geojson',
    'data/geometria/north_america.geojson',
    'data/geometria/oceania.geojson',
    'data/geometria/south_america.geojson'
]

# Crear un diccionario de mapeo global basado en códigos ISO
country_mapping = {}

# Leer todos los GeoJSON y construir el mapeo
for geojson_file in geojson_files:
    if os.path.exists(geojson_file):
        with open(geojson_file, 'r', encoding='utf-8') as f:
            geojson = json.load(f)
            
        for feature in geojson['features']:
            props = feature['properties']
            iso_a3 = props.get('iso_a3')
            standard_name = props.get('name')
            
            if iso_a3 and standard_name:
                country_mapping[iso_a3] = standard_name

print(f"✓ Mapeo desde GeoJSON: {len(country_mapping)} países")

# Mapeo manual para países que faltan o tienen nombres diferentes
manual_mapping = {
    'BRA': 'Brazil',
    'COL': 'Colombia',
    'PER': 'Peru',
    'BOL': 'Bolivia',
    'ARG': 'Argentina',
    'VEN': 'Venezuela',
    'ECU': 'Ecuador',
    'CHL': 'Chile',
    'URY': 'Uruguay',
    'PRY': 'Paraguay',
    'HKG': 'Hong Kong',
    'REU': 'Réunion',
    'TON': 'Tonga',
    'VCT': 'St. Vin. and Gren.',
    'GUY': 'Guyana',
    'BRB': 'Barbados',
    'CYM': 'Cayman Is.',
    'MUS': 'Mauritius',
    'FSM': 'Micronesia',
    'COM': 'Comoros',
    'GLP': 'Guadeloupe',
    'MTQ': 'Martinique',
    'MYT': 'Mayotte',
    'NCL': 'New Caledonia',
    'PYF': 'Fr. Polynesia',
    'SXM': 'Sint Maarten',
    'MAF': 'St-Martin',
    'BLM': 'St-Barthélemy',
    'VGB': 'British Virgin Is.',
    'VIR': 'U.S. Virgin Is.',
    'ASM': 'American Samoa',
    'GUM': 'Guam',
    'MNP': 'N. Mariana Is.',
    'PRI': 'Puerto Rico',
    'BMU': 'Bermuda',
    'MSR': 'Montserrat',
    'TCA': 'Turks and Caicos Is.',
    'AIA': 'Anguilla',
    'GUF': 'French Guiana',
    'SUR': 'Suriname',
    'NIU': 'Niue',
    'COK': 'Cook Is.',
    'WLF': 'Wallis and Futuna',
    'TKL': 'Tokelau',
    'PLW': 'Palau',
    'STP': 'São Tomé and Principe',
    'CPV': 'Cape Verde',
    'SHN': 'St. Helena',
    'MAC': 'Macao',
    'SGP': 'Singapore',
    'LIE': 'Liechtenstein',
    'QAT': 'Qatar',
    'KWT': 'Kuwait',
    'BHR': 'Bahrain',
    'PSE': 'Palestine',
    'TWN': 'Taiwan',
    'PRK': 'North Korea',
    'KOR': 'South Korea',
    # Países pequeños faltantes
    'LCA': 'St. Lucia',
    'DMA': 'Dominica',
    'WSM': 'Samoa',
    'MHL': 'Marshall Is.',
    'GRD': 'Grenada',
    'ATG': 'Antigua and Barb.',
    'SYC': 'Seychelles',
    'TUV': 'Tuvalu',
    'MDV': 'Maldives',
    'KIR': 'Kiribati',
    'KNA': 'St. Kitts and Nevis',
    'MLT': 'Malta'
}

# Combinar ambos mapeos (manual sobrescribe GeoJSON si hay conflicto)
country_mapping.update(manual_mapping)

print(f"Mapeo total con manual: {len(country_mapping)} países")

# 1. Estandarizar emdat-data-processed.csv
print("\n" + "="*80)
print("Estandarizando emdat-data-processed.csv...")
print("="*80)
df_emdat = pd.read_csv('data/emdat-data-processed.csv')
print(f"Países únicos antes: {df_emdat['Country Name'].nunique()}")

for iso_code, standard_name in country_mapping.items():
    df_emdat.loc[df_emdat['ISO'] == iso_code, 'Country Name'] = standard_name

df_emdat.to_csv('data/emdat-data-processed.csv', index=False)
print(f"✓ Archivo actualizado")
print(f"Países únicos después: {df_emdat['Country Name'].nunique()}")

# 2. Estandarizar disasters_by_country.csv
print("\n" + "="*80)
print("Estandarizando disasters_by_country.csv...")
print("="*80)
df_disasters = pd.read_csv('data/disasters_by_country.csv')
print(f"Países únicos antes: {df_disasters['Country Name'].nunique()}")

for iso_code, standard_name in country_mapping.items():
    df_disasters.loc[df_disasters['ISO'] == iso_code, 'Country Name'] = standard_name

df_disasters.to_csv('data/disasters_by_country.csv', index=False)
print(f"Archivo actualizado")
print(f"Países únicos después: {df_disasters['Country Name'].nunique()}")

# Verificar países sin mapear
unmapped = df_disasters[~df_disasters['ISO'].isin(country_mapping.keys())][['ISO', 'Country Name']].drop_duplicates()
if len(unmapped) > 0:
    print(f"Países aún sin mapear ({len(unmapped)}):")
    print(unmapped.to_string(index=False))
else:
    print("\n✓ ¡Todos los países han sido mapeados correctamente!")

print("\n" + "="*80)
print("RESUMEN FINAL")
print("="*80)
print(f"Total países mapeados: {len(country_mapping)}")
print(f"Total países únicos en dataset: {df_disasters['Country Name'].nunique()}")
print(f"Países sin mapear: {len(unmapped)}")
print("Primeros 30 países estandarizados:")
print(df_disasters[['ISO', 'Country Name', 'Region']].drop_duplicates().sort_values('Country Name').head(30).to_string(index=False))


✓ Mapeo desde GeoJSON: 160 países
Mapeo total con manual: 222 países

Estandarizando emdat-data-processed.csv...
Países únicos antes: 217
✓ Archivo actualizado
Países únicos después: 217

Estandarizando disasters_by_country.csv...
Países únicos antes: 217
Archivo actualizado
Países únicos después: 217

✓ ¡Todos los países han sido mapeados correctamente!

RESUMEN FINAL
Total países mapeados: 222
Total países únicos en dataset: 217
Países sin mapear: 0
Primeros 30 países estandarizados:
ISO       Country Name        Region
AFG        Afghanistan          Asia
ALB            Albania        Europe
DZA            Algeria        Africa
ASM     American Samoa       Oceania
AGO             Angola        Africa
AIA           Anguilla South America
ATG  Antigua and Barb. South America
ARG          Argentina South America
ARM            Armenia          Asia
AUS          Australia       Oceania
AUT            Austria        Europe
AZE         Azerbaijan          Asia
BHS            Bahamas South

In [33]:
# Leer el archivo CSV
df = pd.read_csv('data/disasters_by_country.csv')

# Renombrar las columnas
df = df.rename(columns={
    'Country Name': 'Country',
    'Disaster Subgroup': 'disaster',
    'Disaster_Count': 'disaster_count'
})

# Agregar la columna color_intensity igual a disaster_count
df['color_intensity'] = df['disaster_count']

# Obtener las regiones únicas
regions = df['Region'].unique()

# Crear un dataset para cada región/continente
for region in regions:
    # Filtrar datos por región y seleccionar solo las columnas necesarias
    region_df = df[df['Region'] == region][['Country', 'disaster', 'disaster_count', 'color_intensity']]
    
    # Crear nombre de archivo basado en la región
    filename = f"data/disasters_{region.lower().replace(' ', '_')}.csv"
    
    # Guardar el dataset
    region_df.to_csv(filename, index=False)
    print(f"✓ Creado: {filename} ({len(region_df)} registros)")

print(f"\n✓ Se han creado {len(regions)} datasets, uno por continente")
print(f"Continentes procesados: {', '.join(sorted(regions))}")

✓ Creado: data/disasters_north_america.csv (15 registros)
✓ Creado: data/disasters_asia.csv (267 registros)
✓ Creado: data/disasters_south_america.csv (199 registros)
✓ Creado: data/disasters_africa.csv (263 registros)
✓ Creado: data/disasters_europe.csv (180 registros)
✓ Creado: data/disasters_oceania.csv (76 registros)

✓ Se han creado 6 datasets, uno por continente
Continentes procesados: Africa, Asia, Europe, North America, Oceania, South America


In [35]:
# Crear dataset mundial con todos los países
df_world = pd.read_csv('data/disasters_by_country.csv')

# Renombrar las columnas y agregar color_intensity
df_world = df_disasters.rename(columns={
    'Country Name': 'Country',
    'Disaster Subgroup': 'disaster',
    'Disaster_Count': 'disaster_count'
})

# Agregar la columna color_intensity igual a disaster_count
df_world['color_intensity'] = df['disaster_count']

# Seleccionar solo las columnas necesarias
df_world = df[['Country', 'disaster', 'disaster_count', 'color_intensity']]

# Guardar el dataset mundial
df_world.to_csv('data/disasters_world.csv', index=False)
print(f"✓ Creado: disasters_world.csv ({len(df_world)} registros)")
print(f"Países únicos: {df_world['Country'].nunique()}")
print(f"Tipos de desastres: {df['disaster'].unique()}")

✓ Creado: disasters_world.csv (1000 registros)
Países únicos: 217
Tipos de desastres: ['Meteorological' 'Hydrological' 'Geophysical' 'Climatological'
 'Biological' 'Extra-terrestrial']


In [ ]:
# Crear archivos de información de desastres por continente
df_info = pd.read_csv('data/emdat-data-processed.csv')

# Seleccionar y renombrar las columnas necesarias
df_info = df_info[['Disaster Subgroup', 'Disaster Type', 'Event Name', 'Total Deaths', 'Total Affected', 'Start Date', 'End Date', 'Country Name', 'Region']].copy()

df_info = df_info.rename(columns={
    'Disaster Subgroup': 'disaster',
    'Disaster Type': 'disaster_type',
    'Event Name': 'event_name',
    'Total Deaths': 'deaths',
    'Total Affected': 'affected',
    'Start Date': 'start',
    'End Date': 'end',
    'Country Name': 'Country'
})

# Obtener las regiones únicas
regions = df_info['Region'].unique()

# Crear un archivo CSV para cada continente
for region in regions:
    # Filtrar datos por región y eliminar la columna Region
    region_df = df_info[df_info['Region'] == region].drop(columns=['Region'])
    
    # Crear nombre de archivo basado en la región
    filename = f"data/{region.lower().replace(' ', '_')}_disaster_info.csv"
    
    # Guardar el dataset
    region_df.to_csv(filename, index=False)
    print(f"✓ Creado: {filename} ({len(region_df)} registros)")

print(f"\n✓ Se han creado {len(regions)} archivos de información de desastres")
print(f"Continentes procesados: {', '.join(sorted(regions))}")
print(f"\nColumnas incluidas: disaster, event_name, deaths, affected, start, end, Country")

✓ Creado: data/south_america_disaster_info.csv (1738 registros)
✓ Creado: data/north_america_disaster_info.csv (722 registros)
✓ Creado: data/africa_disaster_info.csv (2225 registros)
✓ Creado: data/europe_disaster_info.csv (1390 registros)
✓ Creado: data/asia_disaster_info.csv (4161 registros)
✓ Creado: data/oceania_disaster_info.csv (386 registros)

✓ Se han creado 6 archivos de información de desastres
Continentes procesados: Africa, Asia, Europe, North America, Oceania, South America

Columnas incluidas: disaster, event_name, deaths, affected, start, end, Country
